<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_abril_2020/blob/master/04_ANN_fotos_testa_variaveis_duas_classes_avalia_mar_27_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# rede neural com duas classes grao=2 e nao grao=1+3

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import math
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
import re
%matplotlib inline

In [3]:
!git clone https://github.com/ucfilho/marquesgabi_mar_2020 #clonar do Github
%cd marquesgabi_mar_2020

!git clone https://github.com/marquesgabi/Doutorado
%cd Doutorado

Cloning into 'marquesgabi_mar_2020'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 66 (delta 34), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (66/66), done.
/content/marquesgabi_mar_2020
Cloning into 'Doutorado'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 223 (delta 33), reused 0 (delta 0), pack-reused 147
Receiving objects: 100% (223/223), 87.41 MiB | 26.44 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/marquesgabi_mar_2020/Doutorado


In [0]:
#!ls

In [0]:
#@title Digite os parametros { run: "auto", vertical-output: true }
#@markdown Enter neurons hidden layer
N_Hidden=22  #@param {type:"number"}
#@markdown Activation function

Ativa='tanh'#@param ["tanh", "identity", "logistic","relu"] {allow-input: true}

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [8]:
df_contraste= pd.read_csv('Contraste_Completo_mar_2020.csv') 
#df_contraste= np.delete(df_contraste, 0, axis=1) # delete first colunm
del df_contraste['Unnamed: 0']
Nomes=list(df_contraste.columns)
#del df_contraste['Unnamed: 0'] # delete first colunm
print(Nomes)
print(df_contraste.head())
print(df_contraste.shape)

['Type', 'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors']
   Type      ASM  constrast   correl  ...  dif entropy     IMC1     IMC2  colors
0     1  0.05766   93.16340  0.15346  ...      2.72958 -0.64142  0.98102  0.4125
1     1  0.05013  402.45588  0.32031  ...      2.88069 -0.79959  0.99822  0.4100
2     1  0.05287  154.89461  0.28263  ...      2.93133 -0.74008  0.99509  0.4225
3     1  0.05011  397.55801  0.21760  ...      2.99470 -0.78154  0.99769  0.5300
4     1  0.05408  108.01144  0.34926  ...      2.80948 -0.72303  0.99423  0.4900

[5 rows x 15 columns]
(720, 15)


In [9]:
df_fracionado= pd.read_csv('Contraste_Fracionado_mar_2020.csv') 
del df_fracionado['Unnamed: 0']
Nomes_fracionado=list(df_fracionado.columns)
print(df_fracionado)
#df_fracionado= np.delete(df_fracionado, 0, axis=1) # delete first colunm
#del df_fracionado['Unnamed: 0'] # delete first colunm
#del df_fracionado['Type'] # delete first colunm

print(df_fracionado.head())
#print(df_fracionado.shape)

     N_peaks  Media_proem        Area  ...      Mode       Mean          Sd
0          2     13.64931  1407.70370  ...   7.52778   93.16340   101.41799
1          2    268.50231  6954.06944  ...  36.37500  402.45588   374.61667
2          2     16.58218  2372.18981  ...  46.93750  154.89461   109.03101
3          2      6.75694  5663.77546  ...  77.49306  397.55801   451.88121
4          3     25.91667  1681.69907  ...  10.22917  108.01144    94.42290
..       ...          ...         ...  ...       ...        ...         ...
715        2      2.82755  2325.76157  ...  10.03472  185.27614   346.45698
716        2     10.04282  1243.38426  ...   8.18056   88.03431    95.03825
717        3     12.63426   466.21296  ...   6.06944   29.09273    21.04542
718        2     21.59028  4145.96991  ...  24.84028  563.94812  1900.11526
719        0      0.00000  5489.27083  ...  19.20139  405.43096   487.97502

[720 rows x 10 columns]
   N_peaks  Media_proem        Area  ...      Mode       Mean  

In [10]:
#print(Nomes_contraste)
print(Nomes_fracionado)

['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd']


In [11]:
#ANN_Completo é o dataframe que tem todas as variaveis juntas

ANN_Completo = pd.concat([df_contraste, df_fracionado], axis=1)
print(ANN_Completo.head())
print(ANN_Completo.shape)

   Type      ASM  constrast   correl  ...     Median      Mode       Mean         Sd
0     1  0.05766   93.16340  0.15346  ...   53.48611   7.52778   93.16340  101.41799
1     1  0.05013  402.45588  0.32031  ...  256.77083  36.37500  402.45588  374.61667
2     1  0.05287  154.89461  0.28263  ...  120.70833  46.93750  154.89461  109.03101
3     1  0.05011  397.55801  0.21760  ...  149.81944  77.49306  397.55801  451.88121
4     1  0.05408  108.01144  0.34926  ...   66.18750  10.22917  108.01144   94.42290

[5 rows x 25 columns]
(720, 25)


In [12]:
#Aqui iremos escolher quais variáveis iremos utilizar na ANN

Colunms=['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode','Mean', 'Sd']

#Colunms=['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 
         # 'Width_peaks_max', 'Width_peaks_min', 'Median', 
          #'Mode', 'Mean', 'Sd']

#'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors'
#'N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd'


del df_contraste['Type'] # delete first colunm

Nomes_contraste=list(df_contraste.columns)
for ii in Nomes_contraste:
  Colunms.append(ii)
#X=df.iloc[:,2:12]
#y=df.iloc[:,1].astype(int)
X=ANN_Completo[Colunms]
print(X)
#y=df['Type'].astype(int)
y=ANN_Completo['Type']
print(y)

     N_peaks  Media_proem        Area  ...     IMC1     IMC2  colors
0          2     13.64931  1407.70370  ... -0.64142  0.98102  0.4125
1          2    268.50231  6954.06944  ... -0.79959  0.99822  0.4100
2          2     16.58218  2372.18981  ... -0.74008  0.99509  0.4225
3          2      6.75694  5663.77546  ... -0.78154  0.99769  0.5300
4          3     25.91667  1681.69907  ... -0.72303  0.99423  0.4900
..       ...          ...         ...  ...      ...      ...     ...
715        2      2.82755  2325.76157  ... -0.72472  0.99488  0.5300
716        2     10.04282  1243.38426  ... -0.76055  0.99696  0.4400
717        3     12.63426   466.21296  ... -0.71517  0.99219  0.5225
718        2     21.59028  4145.96991  ... -0.71082  0.99428  0.5325
719        0      0.00000  5489.27083  ... -0.73096  0.99265  0.5100

[720 rows x 24 columns]
0      1
1      1
2      1
3      1
4      1
      ..
715    3
716    3
717    3
718    3
719    3
Name: Type, Length: 720, dtype: int64


In [13]:
print(Colunms)

['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd', 'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors']


In [14]:

# separa os dados variaveis de entrada em normatizados e originais
X_OLD=[]
X_OLD=X.copy()
X=Normatiza(X)
#print(X)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
print(y.ravel())

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [16]:
# esta parte encontra os indices do indefinido para depois 
# usar os indicies para unir indefinido e buraco

Ind=[i for i, e in enumerate(y) if e == 3]
print(Ind)

[540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719]


In [17]:
# esta parte une indefinido e buraco no numero 1 e passa a ser uma so 
# classe por enquanto os dados nao estao balanceados 
# tem o dobro da classe 1 que a classe 2
for i in Ind:
  y[i]=1
print(y.ravel())

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                    random_state=25)

In [0]:
# define a rede neural 
#N_Hidden=(18,18,18)


clf = MLPClassifier(solver='lbfgs',activation=Ativa,alpha=1e-5,
                    hidden_layer_sizes=N_Hidden,random_state=0)


In [20]:
# treina a rede neural
clf.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='tanh', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=22, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=clf.predict(X_train)
y_pred=clf.predict(X_test)


In [22]:
# Evaluating the Algorithm
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[86 24]
 [26 80]]
              precision    recall  f1-score   support

           1       0.77      0.78      0.77       110
           2       0.77      0.75      0.76       106

    accuracy                           0.77       216
   macro avg       0.77      0.77      0.77       216
weighted avg       0.77      0.77      0.77       216



In [23]:
'''
[[31  5 15]
 [ 8 41 11]
 [12 12 27]]
              precision    recall  f1-score   support

           1       0.61      0.61      0.61        51
           2       0.71      0.68      0.69        60
           3       0.51      0.53      0.52        51

    accuracy                           0.61       162
   macro avg       0.61      0.61      0.61       162
weighted avg       0.61      0.61      0.61       162
'''


'\n[[31  5 15]\n [ 8 41 11]\n [12 12 27]]\n              precision    recall  f1-score   support\n\n           1       0.61      0.61      0.61        51\n           2       0.71      0.68      0.69        60\n           3       0.51      0.53      0.52        51\n\n    accuracy                           0.61       162\n   macro avg       0.61      0.61      0.61       162\nweighted avg       0.61      0.61      0.61       162\n'

In [24]:
print(y_pred)

[1 1 1 2 1 2 1 1 2 1 1 2 2 2 2 1 2 1 2 1 1 2 1 2 2 2 2 2 1 1 1 1 1 1 2 1 2
 2 2 2 1 2 2 1 1 2 1 2 1 2 1 2 2 1 2 2 1 1 1 2 1 1 1 1 1 1 2 1 2 2 2 1 1 2
 2 1 1 1 1 2 1 1 2 1 2 1 1 2 2 1 2 1 2 1 2 2 1 2 1 2 2 1 1 2 1 1 2 2 2 2 1
 2 1 1 2 2 2 2 2 2 2 2 2 2 1 2 1 1 2 2 1 1 2 1 1 2 2 1 2 1 2 1 1 2 1 2 2 1
 1 2 1 2 1 1 2 1 1 2 1 2 1 2 1 2 2 1 2 2 1 1 2 1 2 2 1 1 1 2 2 2 2 1 1 1 1
 2 1 2 1 1 1 1 2 2 1 2 2 2 2 1 2 2 2 1 1 1 1 2 1 1 1 2 1 1 2 1]
